In [42]:
import os
import keras
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
import numpy as np

In [43]:
os.makedirs("Images/Class", exist_ok=True)
os.makedirs("Images/output_images", exist_ok=True)

In [44]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)
train_datagen.directory = "../Images/Class/"

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/path/to/training/data/folder",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
)

In [45]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [46]:
vgg16 = VGG16(weights="imagenet", include_top=False)
for layer in vgg16.layers:
    layer.trainable = False

In [47]:
x = vgg16.output
print(x.shape)
x = Flatten(name="flatten")(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation="softmax")(x)


(None, None, None, 512)


ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [ ]:
model = Model(vgg16.input, predictions)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit_generator(
    train_datagen.flow_from_directory("train", target_size=(224, 224), batch_size=32),
    steps_per_epoch=100,
    epochs=10,
    validation_data=test_datagen.flow_from_directory("test", target_size=(224, 224), batch_size=32),
)

In [ ]:
model.save("face_recognition_model.h5")

In [ ]:
model = keras.models.load_model("face_recognition_model.h5")

test_data = test_datagen.flow_from_directory("test", target_size=(224, 224), batch_size=1)

predictions = model.predict(test_data)

for i in range(len(predictions)):
    print(test_data.class_indices[np.argmax(predictions[i])])